<a href="https://colab.research.google.com/github/sdlee1026/machine/blob/master/%EB%94%A5%EB%9F%AC%EB%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 데이터 로드 (파일 경로는 실제 데이터셋의 위치에 따라 다르게 설정해야 합니다.)
data = pd.read_csv('to성대.csv')
np.set_printoptions(suppress=True, precision=10)

In [2]:
##########################################################
# 입력 인자
##########################################################
user_BallSpeed = 61.47

local_ballsx =  512.623535
local_ballsy =  66.5597534
local_ballsz = 405.468231

local_windX = 1.53599513
local_windY = -3.40649748


In [ ]:
# 전처리, 모델 정의 및 피팅

##########################################################
# 전처리 IP 시점 예측 모델
##########################################################

# BallSpeed 는 유저의 평균 데이터를 가져와서 쓸 예정
# 평균 발사각(좌, 우 분리) 사이드 스핀(좌, 우 분리) 가져와서 사용한다면?
lookX = data[['ballsx', 'ballsy', 'ballsz', 'Wind_x', 'Wind_y', 'BallSpeed']]
lookY = data[['Look_x','Look_z']]

lookX_df = []
lookY_df = []
lookYindex = 0

for data_back in data['BackSpin']:
  if data_back != 0:
    lookX_df.append(lookX.loc[lookYindex])
    lookY_df.append(lookY.loc[lookYindex])
  lookYindex += 1

lookX_df_df = pd.DataFrame(lookX_df, columns=['ballsx', 'ballsy', 'ballsz', 'Wind_x', 'Wind_y', 'BallSpeed'])
lookY_df_df = pd.DataFrame(lookY_df, columns=['Look_x','Look_z'])

# 학습 데이터와 테스트 데이터 분리
lookX_train, lookX_test, looky_train, looky_test = train_test_split(lookX_df_df, lookY_df_df, test_size=0.2)

# 데이터 정규화 (Standard Scaling)
scaler_lookX = StandardScaler()
scaler_lookX.fit(lookX_train)

X_train_scaledlook = scaler_lookX.transform(lookX_train)
X_test_scaledlook = scaler_lookX.transform(lookX_test)


# 딥러닝 모델 구성 및 컴파일
model_look = Sequential()
model_look.add(Dense(32, input_dim=lookX.shape[1], activation='relu'))  # 입력층 (32개 노드)
model_look.add(Dense(64, activation='relu'))  # 은닉층 (64개 노드)
model_look.add(Dense(lookY.shape[1]))  # 출력층 (출력 변수의 개수만큼 노드)

model_look.compile(loss='mean_squared_error', optimizer='adam')

# 학습 데이터로 모델 학습
model_look.fit(X_train_scaledlook, looky_train, epochs=250)

# 테스트 데이터로 성능 평가
mse_test_look = model_look.evaluate(X_test_scaledlook , looky_test)
print("Test_look set MSE: ", mse_test_look)

##########################################################
# 센서값 예측 모델
##########################################################
sensorX = data[['ballsx', 'ballsy', 'ballsz', 'Wind_x', 'Wind_y', 'BallSpeed', 'Look_x','Look_z']]
sensorY = data[['Azimuth','BackSpin','SideSpin']]

# '파' 스코어를 기준으로 하기 때문에, 스코어 가중치 사용 x
# weights = 1 / (1 + data['스코어'] + 1e-10)

sensorX_right = []
sensorY_right = [] # sidespin > 0
# w_right = []
sensorX_left = []
sensorY_left = [] #sidespin <= 0
# w_left = []
sensorX_putt = []
sensorY_putt = []
# w_putt = []

y_index = 0

for y_side in sensorY['SideSpin']:
  if sensorY.loc[y_index]['BackSpin'] != 0:
    if int(y_side) > 0:
      sensorX_right.append(sensorX.loc[y_index])
      sensorY_right.append(sensorY.loc[y_index])
      # w_right.append(weights.loc[y_index])
    else:
      sensorX_left.append(sensorX.loc[y_index])
      sensorY_left.append(sensorY.loc[y_index])
      # w_left.append(weights.loc[y_index])
  else:
    sensorX_putt.append(sensorX.loc[y_index])
    sensorY_putt.append(sensorY.loc[y_index])
    # w_putt.append(weights.loc[y_index])
  y_index += 1


sensorX_side_right = pd.DataFrame(sensorX_right, columns=['ballsx', 'ballsy', 'ballsz', 'Wind_x', 'Wind_y', 'BallSpeed','Look_x','Look_z'])
sensorY_side_right = pd.DataFrame(sensorY_right, columns=['Azimuth','BackSpin','SideSpin'])
sensorX_side_left = pd.DataFrame(sensorX_left, columns=['ballsx', 'ballsy', 'ballsz', 'Wind_x', 'Wind_y', 'BallSpeed','Look_x','Look_z'])
sensorY_side_left = pd.DataFrame(sensorY_left, columns=['Azimuth','BackSpin','SideSpin'])
sensorX_side_putt = pd.DataFrame(sensorX_putt, columns=['ballsx', 'ballsy', 'ballsz', 'Wind_x', 'Wind_y', 'BallSpeed','Look_x','Look_z'])
sensorY_side_putt = pd.DataFrame(sensorY_putt, columns=['Azimuth','BackSpin','SideSpin'])

# 학습 데이터와 테스트 데이터 분리

sensorX_train_R, sensorX_test_R, sensorY_train_R, sensorY_test_R = train_test_split(sensorX_side_right, sensorY_side_right, test_size=0.2)
sensorX_train_L, sensorX_test_L, sensorY_train_L, sensorY_test_L = train_test_split(sensorX_side_left, sensorY_side_left, test_size=0.2)
sensorX_train_P, sensorX_test_P, sensorY_train_P, sensorY_test_P = train_test_split(sensorX_side_putt, sensorY_side_putt, test_size=0.2)

# 데이터 정규화 (Standard Scaling)
scaler_sensorX_R = StandardScaler()
scaler_sensorX_R.fit(sensorX_train_R)

sensorX_train_scaled_R = scaler_sensorX_R.transform(sensorX_train_R)
sensorX_test_scaled_R = scaler_sensorX_R.transform(sensorX_test_R)

scaler_sensorX_L = StandardScaler()
scaler_sensorX_L.fit(sensorX_train_L)

sensorX_train_scaled_L = scaler_sensorX_L.transform(sensorX_train_L)
sensorX_test_scaled_L = scaler_sensorX_L.transform(sensorX_test_L)


# 딥러닝 모델 구성 및 컴파일
sensor_model_L = Sequential()
sensor_model_L.add(Dense(32, input_dim=sensorX.shape[1], activation='relu'))  # 입력층 (32개 노드)
sensor_model_L.add(Dense(64, activation='relu'))  # 은닉층 (64개 노드)
sensor_model_L.add(Dense(sensorY.shape[1]))  # 출력층 (출력 변수의 개수만큼 노드)

sensor_model_R = Sequential()
sensor_model_R.add(Dense(32, input_dim=sensorX.shape[1], activation='relu'))  # 입력층 (32개 노드)
sensor_model_R.add(Dense(64, activation='relu'))  # 은닉층 (64개 노드)
sensor_model_R.add(Dense(sensorY.shape[1]))  # 출력층 (출력 변수의 개수만큼 노드)

sensor_model_L.compile(loss='mean_squared_error', optimizer='adam')

# 학습 데이터로 모델 학습
history = sensor_model_L.fit(sensorX_train_scaled_L, sensorY_train_L, epochs=250)

# 테스트 데이터로 성능 평가
sensor_mse_test_L = sensor_model_L.evaluate(sensorX_test_scaled_L , sensorY_test_L)
print("Test_L set MSE: ", sensor_mse_test_L)

###################
sensor_model_R.compile(loss='mean_squared_error', optimizer='adam')

# 학습 데이터로 모델 학습
history = sensor_model_R.fit(sensorX_train_scaled_R, sensorY_train_R, epochs=250)

# 테스트 데이터로 성능 평가
sensor_mse_test_R = sensor_model_R.evaluate(sensorX_test_scaled_R , sensorY_test_R)
print("Test_R set MSE: ", sensor_mse_test_R)

##########################################################
# 최적 IP 예측 모델
##########################################################
after_lookX = data[['ballsx', 'ballsy', 'ballsz', 'Wind_x', 'Wind_y', 'BallSpeed', 'Azimuth', 'BackSpin', 'SideSpin']]
# 앞전 예측값들 활용
after_lookY = data[['Look_x','Look_z']]

after_lookX_df = []
after_lookY_df = []
after_lookYindex = 0

for data_back in data['BackSpin']:
  if data_back != 0:
    after_lookX_df.append(after_lookX.loc[after_lookYindex])
    after_lookY_df.append(after_lookY.loc[after_lookYindex])
  after_lookYindex += 1

after_lookX_df_df = pd.DataFrame(after_lookX_df, columns=['ballsx', 'ballsy', 'ballsz', 'Wind_x', 'Wind_y', 'BallSpeed', 'Azimuth', 'BackSpin', 'SideSpin'])
after_lookY_df_df = pd.DataFrame(after_lookY_df, columns=['Look_x','Look_z'])

after_lookX_train, after_lookX_test, after_looky_train, after_looky_test = train_test_split(after_lookX_df_df, after_lookY_df_df, test_size=0.2)

# 데이터 정규화 (Standard Scaling)
after_scaler_lookX = StandardScaler()
after_scaler_lookX.fit(after_lookX_train)

after_X_train_scaledlook = after_scaler_lookX.transform(after_lookX_train)
after_X_test_scaledlook = after_scaler_lookX.transform(after_lookX_test)


# 딥러닝 모델 구성 및 컴파일
after_model_look = Sequential()
after_model_look.add(Dense(32, input_dim=after_lookX.shape[1], activation='relu'))  # 입력층 (32개 노드)
after_model_look.add(Dense(64, activation='relu'))  # 은닉층 (64개 노드)
after_model_look.add(Dense(after_lookY.shape[1]))  # 출력층 (출력 변수의 개수만큼 노드)

after_model_look.compile(loss='mean_squared_error', optimizer='adam')

# 학습 데이터로 모델 학습
history = after_model_look.fit(after_X_train_scaledlook, after_looky_train, epochs=250)

# 테스트 데이터로 성능 평가
mse_test_after = after_model_look.evaluate(after_X_test_scaledlook , after_looky_test)
print("Test_L set MSE: ", mse_test_after)

Epoch 1/250
196/196 [==============================] - 3s 4ms/step - loss: 0.0970
Epoch 2/250
196/196 [==============================] - 1s 4ms/step - loss: 0.0296
Epoch 3/250
196/196 [==============================] - 1s 6ms/step - loss: 0.0220
Epoch 4/250
196/196 [==============================] - 1s 4ms/step - loss: 0.0185
Epoch 5/250
196/196 [==============================] - 1s 4ms/step - loss: 0.0153
Epoch 6/250
196/196 [==============================] - 1s 5ms/step - loss: 0.0126
Epoch 7/250
196/196 [==============================] - 1s 4ms/step - loss: 0.0102
Epoch 8/250
196/196 [==============================] - 1s 6ms/step - loss: 0.0085
Epoch 9/250
196/196 [==============================] - 1s 3ms/step - loss: 0.0070
Epoch 10/250
196/196 [==============================] - 1s 3ms/step - loss: 0.0063
Epoch 11/250
196/196 [==============================] - 1s 3ms/step - loss: 0.0056
Epoch 12/250
196/196 [==============================] - 1s 3ms/step - loss: 0.0051
Epoch 13/250


In [ ]:
##########################################################
# 예측 전처리 IP
##########################################################
new_data_look = {
    'ballsx': [local_ballsx],
    'ballsy': [local_ballsy],
    'ballsz': [local_ballsz],
    'Wind_x': [local_windX],
    'Wind_y': [local_windY],
    'BallSpeed' : [user_BallSpeed]
}

# 새로운 데이터를 DataFrame으로 변환
new_data_df_look = pd.DataFrame(new_data_look)

# 정규화
new_data_scaled_look = scaler_lookX.transform(new_data_df_look)

# 모델에 새로운 데이터를 넣고 예측
prediction_Look = model_look.predict(new_data_scaled_look)

print("prediction_Look: ", prediction_Look)

print("start locaiton\n:" + str(new_data_look))

print("\nshot path loging")
print("\npre_Look_x : " + str(prediction_Look[0][0])
+ "\npre_Look_z : " + str(prediction_Look[0][1])
)
# 'Look_x','Look_z'

pre_lookX = prediction_Look[0][0]
pre_lookY = prediction_Look[0][1]


In [ ]:

##########################################################
# 예측 센서 값
##########################################################
new_data_sensor = {
    'ballsx': [local_ballsx],
    'ballsy': [local_ballsy],
    'ballsz': [local_ballsz],
    'Wind_x': [local_windX],
    'Wind_y': [local_windY],
    'BallSpeed' : [user_BallSpeed],
    'Look_x' : [pre_lookX],
    'Look_z' : [pre_lookY]
}

# 새로운 데이터를 DataFrame으로 변환
new_data_df_sensor = pd.DataFrame(new_data_sensor)

# 정규화
new_data_scaled_sensor_L = scaler_sensorX_L.transform(new_data_df_sensor)
new_data_scaled_sensor_R = scaler_sensorX_R.transform(new_data_df_sensor)

# 모델에 새로운 데이터를 넣고 예측
prediction_sensor_L = sensor_model_L.predict(new_data_scaled_sensor_L)

print("Prediction_sensor_L : ", prediction_sensor_L)

# 모델에 새로운 데이터를 넣고 예측
prediction_sensor_R = sensor_model_R.predict(new_data_scaled_sensor_R)

print("prediction_sensor_R : ", prediction_sensor_R)

print("start locaiton\n:" + str(new_data_sensor))

print("\nshot path loging")
print("\nSideSpin Left Log\nAzimuth : ", str(prediction_sensor_L[0][0])
+ "\nBackSpin : " + str(prediction_sensor_L[0][1])
+ "\nSideSpin : " + str(prediction_sensor_L[0][2])
)

print("\nSideSpin Right Log\nAzimuth : ", str(prediction_sensor_R[0][0])
+ "\nBackSpin : " + str(prediction_sensor_R[0][1])
+ "\nSideSpin : " + str(prediction_sensor_R[0][2])
)

pre_Azi_L = prediction_sensor_L[0][0]
pre_Back_L = prediction_sensor_L[0][1]
pre_Side_L = prediction_sensor_L[0][2]

pre_Azi_R = prediction_sensor_R[0][0]
pre_Back_R = prediction_sensor_R[0][1]
pre_Side_R = prediction_sensor_R[0][2]

In [ ]:
new_data_L = {
    'ballsx': [local_ballsx],
    'ballsy': [local_ballsy],
    'ballsz': [local_ballsz],
    'Wind_x': [local_windX],
    'Wind_y': [local_windY],
    'BallSpeed' : [user_BallSpeed],
    'Azimuth' : [pre_Azi_L],
    'BackSpin' : [pre_Back_L],
    'SideSpin' : [pre_Side_L]
}
new_data_R = {
    'ballsx': [local_ballsx],
    'ballsy': [local_ballsy],
    'ballsz': [local_ballsz],
    'Wind_x': [local_windX],
    'Wind_y': [local_windY],
    'BallSpeed' : [user_BallSpeed],
    'Azimuth' : [pre_Azi_R],
    'BackSpin' : [pre_Back_R],
    'SideSpin' : [pre_Side_R]
}

# 새로운 데이터를 DataFrame으로 변환
new_data_df_L = pd.DataFrame(new_data_L)
new_data_df_R = pd.DataFrame(new_data_R)

# 정규화
new_data_scaled_L = after_scaler_lookX.transform(new_data_df_L)
new_data_scaled_R = after_scaler_lookX.transform(new_data_df_R)

# 모델에 새로운 데이터를 넣고 예측
prediction_L = after_model_look.predict(new_data_scaled_L)

print("Prediction_L : ", prediction_L)

prediction_R = after_model_look.predict(new_data_scaled_R)

print("Prediction_R : ", prediction_R)

In [ ]:
print("result look X L : ", prediction_L[0][0])
print("result look Y L : ", prediction_L[0][1])
print("pre_Azi_L : ", pre_Azi_L)
print("pre_Side_L : ", pre_Side_L)
print("pre_Back_L : ", pre_Back_L)
print("\n\n\nresult look X R : ", prediction_R[0][0])
print("result look Y R : ", prediction_R[0][1])
print("pre_Azi_R : ", pre_Azi_R)
print("pre_Side_R : ", pre_Side_R)
print("pre_Back_R : ", pre_Back_R)

In [ ]:
print("user_BallSpeed : ", user_BallSpeed)
print("local_ballsx : ", local_ballsx)
print("local_ballsy : ", local_ballsy)
print("local_ballsz : ", local_ballsz)
print("local_windX : ", local_windX)
print("local_windY : ", local_windY)